In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91909\\Documents\\Term 3\\Machine Learning\\assignment\\senti\\Sentiment-Analysis-App\\research'

In [3]:
%pwd

'c:\\Users\\91909\\Documents\\Term 3\\Machine Learning\\assignment\\senti\\Sentiment-Analysis-App\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\91909\\Documents\\Term 3\\Machine Learning\\assignment\\senti\\Sentiment-Analysis-App'

In [6]:
import pandas as pd

In [27]:
df = pd.read_csv('artifacts/McDonald_s_Reviews_ABS.csv')
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,combinations
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - positive
1,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - neutral
2,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - negative
3,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - conflict
4,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - none


In [33]:
df.combinations.value_counts()

combinations
price - positive        2
food - conflict         2
service - conflict      2
service - negative      2
service - neutral       2
service - positive      2
ambience - none         2
ambience - conflict     2
ambience - negative     2
ambience - neutral      2
ambience - positive     2
food - none             2
food - negative         2
price - neutral         2
food - neutral          2
food - positive         2
anecdotes - none        2
anecdotes - conflict    2
anecdotes - negative    2
anecdotes - neutral     2
anecdotes - positive    2
price - none            2
price - conflict        2
price - negative        2
service - none          2
Name: count, dtype: int64

In [8]:
from sentimentAnalysisApp.constants import data_transformation
from sentimentAnalysisApp.logging import logger
from transformers import BertTokenizer, BertForSequenceClassification

c:\Users\91909\Documents\Term 3\Machine Learning\assignment\senti\Sentiment-Analysis-App\senti\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
df = df[:50]

In [10]:
import torch

tokenizer = BertTokenizer.from_pretrained('pretrained')
model = BertForSequenceClassification.from_pretrained('pretrained')
        
# Ensure model is in evaluation mode
model.eval()

# Assuming you have the tokenizer and model loaded as per your initial setup
# tokenizer = BertTokenizer.from_pretrained('savepath')
# model = BertForSequenceClassification.from_pretrained(f"{base_dir}/models/{dataset_type}/BERT-pair/{task}/last_step")

# Tokenize the input pairs
reviews_list = df['review'].tolist()
combinations_list = df['combinations'].tolist()

test_encodings = tokenizer(reviews_list, combinations_list, truncation=True, padding=True, return_tensors="pt")

# Get raw predictions (logits) from the model
with torch.no_grad():
    logits = model(**test_encodings).logits
logits = logits.numpy()


In [12]:
class ABSA_PredictDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])  # Assuming 'input_ids' is a key in encodings


In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

trainer = Trainer(
        model=model
    )

test_encodings = tokenizer(reviews_list, combinations_list, truncation=True, padding=True, return_tensors="pt")
test_predict_dataset = ABSA_PredictDataset(test_encodings)
results = trainer.predict(test_predict_dataset)


C:\Users\91909\AppData\Local\Temp\ipykernel_24092\3022877000.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7/7 [00:24<00:00,  3.57s/it]


In [20]:
import matplotlib.pyplot as plt
from scipy.special import softmax
import numpy as np

# Predict using the Trainer
scores = [softmax(prediction) for prediction in results.predictions]

# Get the predicted labels
predicted_labels = [np.argmax(x) for x in scores]

# Convert predictions and scores to a DataFrame for visualization
df_results = pd.DataFrame({
    'Predicted_Label': predicted_labels,
    'Scores': scores
})

# Display the first few rows of the DataFrame
df_results.head()

,Predicted_Label,Scores
0,0,"[0.9997243, 0.0002756907]"
1,0,"[0.9998816, 0.000118363736]"
2,0,"[0.9985668, 0.0014332216]"
3,0,"[0.9999032, 9.684815e-05]"
4,1,"[0.00059828005, 0.9994017]"


In [29]:
# append df_results to df
dfcat = pd.concat([df, df_results], axis=1)
dfcat.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,combinations,Predicted_Label,Scores
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - positive,0,"[0.9997243, 0.0002756907]"
1,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - neutral,0,"[0.9998816, 0.000118363736]"
2,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - negative,0,"[0.9985668, 0.0014332216]"
3,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - conflict,0,"[0.9999032, 9.684815e-05]"
4,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,price - none,1,"[0.00059828005, 0.9994017]"


In [21]:
evaluation_result = trainer.evaluate(test_predict_dataset)
print(evaluation_result)


C:\Users\91909\AppData\Local\Temp\ipykernel_24092\3022877000.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7/7 [00:36<00:00,  5.21s/it]

{'eval_runtime': 43.9105, 'eval_samples_per_second': 1.139, 'eval_steps_per_second': 0.159}


In [22]:
evaluation_result

{'eval_runtime': 43.9105,
 'eval_samples_per_second': 1.139,
 'eval_steps_per_second': 0.159}

In [ ]:
# Assuming `aspects` and `sentiments` lists are defined as before
combinations = [f"{aspect} - {sentiment}" for aspect in aspects for sentiment in sentiments]

# Create a dictionary mapping for numerical predictions to combinations
prediction_map = {i: combo for i, combo in enumerate(combinations)}

# Convert numerical predictions to combinations
df['Predicted_Combination'] = df['Predicted_Label'].map(prediction_map)
